In [1]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [2]:
from sklearn import datasets
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import  pyspark.sql.functions as F
import random
from pyspark.sql.types import *
from sklearn.model_selection import train_test_split

# Create/Load the Dataset

In [4]:
X,y = datasets.make_classification(n_samples=10000, n_features=4, n_informative=2, n_classes=2, random_state=1,shuffle=True)

In [5]:
train = pd.DataFrame(X)
train['target'] = y

In [6]:
train_sp = spark.createDataFrame(train)

In [7]:
train_sp = train_sp.toDF(*['c0', 'c1', 'c2', 'c3', 'target'])

In [8]:
train_sp.show()

+--------------------+--------------------+--------------------+--------------------+------+
 c0| c1| c2| c3|target|
+--------------------+--------------------+--------------------+--------------------+------+
 -1.8873649371603203| -1.1455691898002351| 0.8396761000312767| -2.0088557080867426| 0|
-0.18266809216582025|-0.12226678277057923| 0.08251252435219325| -0.2054662226628986| 0|
 -0.7315948349672106| 0.6559036936823883| 0.20531124349308189| 0.28714060384238044| 0|
 -0.7749652163170958| 0.7440265567629247| 0.21210307064010814| 0.35187466485355684| 0|
 -1.3394227045324436| -1.0424630852864827| 0.6209706165480363| -1.6479989577746343| 0|
-0.18017499772535683| -1.6244897462240875| 0.24568656908689943| -1.6598910893214862| 1|
 0.82509156468382| -0.5728119479422702| -0.2497852096620914|-0.16210051159032351| 0|
 1.1781029789531323| 0.17113092183663403| -0.4647047960767629| 0.7268761264280708| 1|
 -1.2057458102820222| 1.4411211027600361| 0.2990313154948841| 0.8221866229867812| 0|
-0.23066810781360036|-0.18246194798172555| 0.10726079558688517|-0.28665299425314167| 0|
 -1.0072234845517323| 0.8545722498639476| 0.2879547824996853| 0.348380613815614| 0|
 0.47538225794367417| 1.7486569358975648| -0.3710120313222654| 1.9207939550154742| 1|
 -0.2131515767790706| -1.748906639130869| 0.27176346054110195| -1.7961522233276521| 1|
 -1.5496574364883247| 1.775625586462539| 0.39268549198152913| 0.9825290487330922| 0|
 -1.1086769809797885| 1.3756666418155143| 0.269433691991313| 0.804990744964285| 0|
 1.5405802242318427| -0.7978912930416052|-0.49606651071854874|-0.03945467385592...| 1|
 0.2892416515109355| -0.6920923470206011|-0.03389062031080514| -0.5328709046924676| 0|
 0.29676943127701305| -1.3502191165401722| 0.03515985815615573| -1.1669923954311183| 1|
 1.3199355367253436| 1.4688067175103| -0.6601711724555561| 2.051833559881125| 1|
 -2.125342086258849| 2.3959332555912507| 0.5428607911795114| 1.309428138106237| 0|
+--------------------+--------------------+--------------------+--------------------+------+
only showing top 20 rows

# Replicate the Dataset n times

In [10]:
# replicate the spark dataframe into multiple copies
replication_df = spark.createDataFrame(pd.DataFrame(list(range(1,100)),columns=['replication_id']))

In [11]:
replicated_train_df = train_sp.crossJoin(replication_df)

In [12]:
replicated_train_df.show()

+--------------------+--------------------+-------------------+-------------------+------+--------------+
 c0| c1| c2| c3|target|replication_id|
+--------------------+--------------------+-------------------+-------------------+------+--------------+
 -1.8873649371603203| -1.1455691898002351| 0.8396761000312767|-2.0088557080867426| 0| 1|
 -1.8873649371603203| -1.1455691898002351| 0.8396761000312767|-2.0088557080867426| 0| 2|
 -1.8873649371603203| -1.1455691898002351| 0.8396761000312767|-2.0088557080867426| 0| 3|
 -1.8873649371603203| -1.1455691898002351| 0.8396761000312767|-2.0088557080867426| 0| 4|
 -1.8873649371603203| -1.1455691898002351| 0.8396761000312767|-2.0088557080867426| 0| 5|
 -1.8873649371603203| -1.1455691898002351| 0.8396761000312767|-2.0088557080867426| 0| 6|
 -1.8873649371603203| -1.1455691898002351| 0.8396761000312767|-2.0088557080867426| 0| 7|
 -1.8873649371603203| -1.1455691898002351| 0.8396761000312767|-2.0088557080867426| 0| 8|
 -1.8873649371603203| -1.1455691898002351| 0.8396761000312767|-2.0088557080867426| 0| 9|
 -1.8873649371603203| -1.1455691898002351| 0.8396761000312767|-2.0088557080867426| 0| 10|
 -1.8873649371603203| -1.1455691898002351| 0.8396761000312767|-2.0088557080867426| 0| 11|
 -1.8873649371603203| -1.1455691898002351| 0.8396761000312767|-2.0088557080867426| 0| 12|
 -1.8873649371603203| -1.1455691898002351| 0.8396761000312767|-2.0088557080867426| 0| 13|
-0.18266809216582025|-0.12226678277057923|0.08251252435219325|-0.2054662226628986| 0| 1|
-0.18266809216582025|-0.12226678277057923|0.08251252435219325|-0.2054662226628986| 0| 2|
-0.18266809216582025|-0.12226678277057923|0.08251252435219325|-0.2054662226628986| 0| 3|
-0.18266809216582025|-0.12226678277057923|0.08251252435219325|-0.2054662226628986| 0| 4|
-0.18266809216582025|-0.12226678277057923|0.08251252435219325|-0.2054662226628986| 0| 5|
-0.18266809216582025|-0.12226678277057923|0.08251252435219325|-0.2054662226628986| 0| 6|
-0.18266809216582025|-0.12226678277057923|0.08251252435219325|-0.2054662226628986| 0| 7|
+--------------------+--------------------+-------------------+-------------------+------+--------------+
only showing top 20 rows

# Create Pandas UDF to run Model

In [14]:
# 0. Declare the schema for the output of our function
outSchema = StructType([StructField('replication_id',IntegerType(),True),StructField('Accuracy',DoubleType(),True),StructField('num_trees',IntegerType(),True),StructField('depth',IntegerType(),True),StructField('criterion',StringType(),True)])

# decorate our function with pandas_udf decorator
@F.pandas_udf(outSchema, F.PandasUDFType.GROUPED_MAP)
def run_model(pdf):
    # 1. Get hyperparam values
    num_trees =  random.choice(list(range(50,500)))
    depth = random.choice(list(range(2,10)))
    criterion = random.choice(['gini','entropy'])
    replication_id = pdf.replication_id.values[0]
    # 2. Train test split
    X = pdf[['c0', 'c1', 'c2', 'c3']]
    y = pdf['target']
    #del X['target']
    Xtrain,Xcv,ytrain,ycv = train_test_split(X, y, test_size=0.33, random_state=42)
    # 3. Create model using the pandas dataframe
    clf = RandomForestClassifier(n_estimators=num_trees, max_depth = depth, criterion =criterion)
    clf.fit(Xtrain,ytrain)
    # 4. Evaluate the model
    accuracy = accuracy_score(clf.predict(Xcv),ycv)
    # 5. return results as pandas DF
    res =pd.DataFrame({'replication_id':replication_id,'Accuracy':accuracy, 'num_trees':num_trees,'depth':depth,'criterion':criterion}, index=[0])
    return res



# Run the model

In [16]:
results = replicated_train_df.groupby("replication_id").apply(run_model)
results.sort(F.desc("Accuracy")).show()

+--------------+------------------+---------+-----+---------+
replication_id| Accuracy|num_trees|depth|criterion|
+--------------+------------------+---------+-----+---------+
 81| 0.963030303030303| 220| 9| entropy|
 63| 0.963030303030303| 306| 9| entropy|
 98|0.9627272727272728| 413| 9| gini|
 20|0.9624242424242424| 72| 8| gini|
 50|0.9624242424242424| 371| 9| gini|
 87|0.9624242424242424| 338| 9| entropy|
 9|0.9624242424242424| 159| 8| gini|
 71|0.9624242424242424| 277| 9| entropy|
 42|0.9624242424242424| 438| 9| entropy|
 16|0.9621212121212122| 166| 9| entropy|
 41|0.9621212121212122| 109| 9| entropy|
 66|0.9618181818181818| 337| 8| entropy|
 57|0.9618181818181818| 133| 8| entropy|
 51|0.9618181818181818| 153| 8| gini|
 91|0.9618181818181818| 113| 9| gini|
 13|0.9618181818181818| 57| 7| gini|
 35|0.9615151515151515| 302| 8| gini|
 12|0.9615151515151515| 216| 6| entropy|
 27|0.9615151515151515| 315| 7| gini|
 48|0.9615151515151515| 475| 8| gini|
+--------------+------------------+---------+-----+---------+
only showing top 20 rows